In [21]:
import pandas as pd
from pandas.api import types as pdt

def clean_field_names_pandas(df: pd.DataFrame, case: str = "lower") -> pd.DataFrame:
    """
    Renombra columnas:
      - strip + collapse espacios
      - espacios → "_"
      - quita tildes
      - pasa a lower/upper según case
    """
    cols = (
        df.columns
          .str.strip()
          .str.replace(r"\s+", " ", regex=True)
          .str.replace(" ", "_")
          .str.normalize("NFKD")
          .str.encode("ascii", "ignore").str.decode("ascii")
    )
    cols = cols.str.upper() if case.lower() == "upper" else cols.str.lower()
    df = df.copy()
    df.columns = cols
    return df

def _clean_str_series(ser: pd.Series, case: str) -> pd.Series:
    """
    Limpia una Serie de texto:
      - strip + collapse espacios
      - quita tildes
      - pasa a lower/upper según case
    Usa StringDtype para preservar nulos.
    """
    s = ser.astype("string")
    s = (
        s.str.strip()
         .str.replace(r"\s+", " ", regex=True)
         .str.normalize("NFKD")
         .str.encode("ascii", "ignore").str.decode("ascii")
    )
    s = s.str.upper() if case.lower() == "upper" else s.str.lower()
    return s

def clean_text_values_pandas(df: pd.DataFrame, case: str = "lower") -> pd.DataFrame:
    """
    Normaliza valores de texto en columnas object, string o category.
    """
    df = df.copy()
    for col in df.columns:
        dtype = df[col].dtype

        if isinstance(dtype, pd.CategoricalDtype):
            # limpia categorías y valores
            ser = df[col]
            cats = pd.Series(ser.cat.categories, dtype="string")
            cats_clean = _clean_str_series(cats, case)
            ser_new = ser.cat.set_categories(cats_clean).astype("string")
            ser_new = _clean_str_series(ser_new, case).astype("category")

        elif pdt.is_object_dtype(dtype) or pdt.is_string_dtype(dtype):
            ser_new = _clean_str_series(df[col], case)

        else:
            continue

        df[col] = ser_new
    return df

def clean_datetime_columns_pandas(
    df: pd.DataFrame,
    datetime_params: dict = None
) -> pd.DataFrame:
    """
    Convierte a datetime SOLO las columnas listadas en datetime_params:
      datetime_params = { col_name: format_str, ... }
    No hay inferencia automática.
    """
    df = df.copy()
    datetime_params = datetime_params or {}

    for col, fmt in datetime_params.items():
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format=fmt, errors="coerce")
            print(f"*** Columna {col!r}: convertida con formato {fmt} ***")
    return df

def fill_nulls(df: pd.DataFrame, fill_map: dict) -> pd.DataFrame:
    """
    Rellena nulos según fill_map = { col_name: fill_value, ... }
    """
    df = df.copy()
    for col, val in fill_map.items():
        if col in df.columns:
            before = df[col].isna().sum()
            df[col] = df[col].fillna(val)
            after = df[col].isna().sum()
            print(f"*** Columna {col!r} con {before} nulls → rellenado con {val!r} ({after} nulls actualmente) *** ")
    return df

def check_duplicates(df: pd.DataFrame, subset: list = None, max_display: int = 5) -> None:
    """
    Muestra registros duplicados en el DataFrame.
    
    Parámetros:
    - subset: columnas a considerar para detectar duplicados (por defecto, todas).
    - max_display: cuántos duplicados mostrar (por defecto, 5).
    """
    dup_mask = df.duplicated(subset=subset, keep=False)
    dup_count = dup_mask.sum()
    
    if dup_count == 0:
        print("*** No hay duplicados encontrados ***")
    else:
        cols_info = f"en columnas: {subset}" if subset else "en todas las columnas"
        print(f"*** Se encontraron {dup_count} registros duplicados {cols_info}. Mostrando primeros {max_display}: ***")
        display(df[dup_mask].head(max_display))

import operator

def replace_invalid_numeric_values(df: pd.DataFrame, numeric_check_conf: dict) -> pd.DataFrame:
    df = df.copy()

    ops = {
        "<": operator.lt,
        "<=": operator.le,
        "==": operator.eq,
        "!=": operator.ne,
        ">=": operator.ge,
        ">": operator.gt,
    }

    fields = numeric_check_conf.get("fields_name", [])
    op_str = numeric_check_conf.get("op", "<=")
    threshold = numeric_check_conf.get("value", 0)
    method = numeric_check_conf.get("method", "mean").lower()

    if op_str not in ops:
        print(f"***  Operador {op_str!r} no soportado. ***")
        return df

    op_func = ops[op_str]

    for col in fields:
        if col not in df.columns:
            print(f"*** Columna {col!r} no existe en el DataFrame. ***")
            continue

        mask = op_func(df[col], threshold)
        count = mask.sum()

        if count == 0:
            print(f"*** Columna {col!r}: ningún valor cumple la condición '{col} {op_str} {threshold}' — no se reemplaza nada. ***")
            continue

        if method == "mean":
            replacement = df.loc[~mask, col].mean()
        elif method == "median":
            replacement = df.loc[~mask, col].median()
        elif method == "mode":
            mode = df.loc[~mask, col].mode()
            replacement = mode[0] if not mode.empty else None
        else:
            print(f"*** Método {method!r} no soportado. Se omite columna {col!r}. ***")
            continue

        print(f"*** Columna {col!r}: {count} valores reemplazados con {method} ({replacement:.2f}) usando condición '{col} {op_str} {threshold}'***")
        df.loc[mask, col] = replacement

    return df


def clean_pandas_df(
    df: pd.DataFrame,
    fields_name_params: dict,
    values_params: dict,
) -> pd.DataFrame:
    """
    Pipeline:
      1) Renombra columnas (fields_name_params['case'])
      2) Convierte fechas (fields_name_params['datetime_fields'])
      3) Rellena nulos (values_params['change_nulls_maps'])
      4) Normaliza texto (values_params['case'])
    """
    # 1) Renombrar columnas
    case_names = fields_name_params.get("case", "lower")
    df2 = clean_field_names_pandas(df, case_names)

    # 2) Convertir datetime
    datetime_fields = fields_name_params.get("datetime_fields", {})
    if datetime_fields:
        df2 = clean_datetime_columns_pandas(df2, datetime_fields)

    # 3) Rellenar nulos
    null_fill_map = values_params.get("change_nulls_maps", {})
    if null_fill_map:
        df2 = fill_nulls(df2, null_fill_map)

    # 4) Normalizar texto
    case_values = values_params.get("case", "lower")
    df2 = clean_text_values_pandas(df2, case_values)

    # 5) Comprobar duplicados
    check_duplicates(df2)

    #6) Validación datos numéricos
    numeric_check_params = values_params.get("numeric_checks", {})
    if numeric_check_params:
        df2 = replace_invalid_numeric_values(df2, numeric_check_params)

    return df2




In [22]:
df_customers = pd.read_csv('./../datasets_limpios/olist_order_customer.csv')

df_customers.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [23]:
df_orders = pd.read_csv('./../datasets_limpios/olist_orders_dataset.csv')

print(df_orders.info())
df_orders



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
 8   tiempo_entrega                 96476 non-null  object
dtypes: object(9)
memory usage: 6.8+ MB
None


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,tiempo_entrega
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8 days 10:28:40
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13 days 18:46:08
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9 days 09:27:40
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13 days 05:00:36
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2 days 20:58:23
...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,8 days 05:13:56
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,22 days 04:38:58
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,24 days 20:37:34
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,17 days 02:04:27


In [24]:
fields_name_params = {
    "case": "lower",
    "datetime_fields": {
        "order_purchase_timestamp": "%Y-%m-%d %H:%M:%S",
        "order_approved_at": "%Y-%m-%d %H:%M:%S",
        "order_delivered_carrier_date": "%Y-%m-%d %H:%M:%S",
        "order_delivered_customer_date": "%Y-%m-%d %H:%M:%S",
        "order_estimated_delivery_date": "%Y-%m-%d",
    }
}

values_params = {
    "case": "lower",
}

df_orders = clean_pandas_df(df_orders, fields_name_params, values_params)
print(df_orders.info())
df_orders


*** Columna 'order_purchase_timestamp': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_approved_at': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_delivered_carrier_date': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_delivered_customer_date': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_estimated_delivery_date': convertida con formato %Y-%m-%d ***
*** No hay duplicados encontrados ***
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at       

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,tiempo_entrega
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,8 days 10:28:40
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,13 days 18:46:08
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,9 days 09:27:40
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,13 days 05:00:36
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2 days 20:58:23
...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,8 days 05:13:56
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,22 days 04:38:58
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,24 days 20:37:34
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,17 days 02:04:27


In [25]:
df_merged = df_customers.merge(df_orders, on='customer_id', how='left')
df_merged

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,tiempo_entrega
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,sp,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,8 days 19:30:00
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,sp,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,16 days 15:52:55
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,sp,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,26 days 01:51:06
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,sp,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,14 days 23:57:47
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,sp,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,11 days 11:04:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,sp,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25,6 days 04:18:20
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,sp,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20,7 days 10:34:23
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,ce,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02,30 days 22:51:25
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,rs,e31ec91cea1ecf97797787471f98a8c2,delivered,2017-11-03 21:08:33,2017-11-03 21:31:20,2017-11-06 18:24:41,2017-11-16 19:58:39,2017-12-05,12 days 22:50:06


In [56]:
# Mergeo ambos dataset quedandome con todos los registros de ambos, luego filtraré para el objetivo del ejercicio

df_merged2 = df_customers.merge(df_orders, on='customer_id', how='outer', indicator=True)

df_merged2[df_merged2['order_status'].isnull()].count()

print("*"*50)
print(df_merged2[df_merged2['_merge'] == 'both'].shape)
print("*"*50)
df_merged2.info()
print("*"*50)
#Vemos si hay duplicados de customer_id que indicará que hay clientes con varios pedidos registrados
print(df_merged2['customer_id'].value_counts()[df_merged2['customer_id'].value_counts() > 1].shape)

#Como vemos todos los registros se han unido como both, por lo que no hay registros que estén en un lado y no en otro
# Además, en el df de orers no hay ningún cliente repetido, es decir, no tenemos clientes que hayan hecho más de un pedido,
#  por lo que no tenemos que agrupar por cliente y quedarnos con la fecha del último pedido realizado, ya que solamente ha realizado uno

**************************************************
(99441, 13)
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   customer_id                    99441 non-null  object  
 1   customer_unique_id             99441 non-null  object  
 2   customer_zip_code_prefix       99441 non-null  int64   
 3   customer_city                  99441 non-null  object  
 4   customer_state                 99441 non-null  object  
 5   order_id                       99441 non-null  object  
 6   order_status                   99441 non-null  object  
 7   order_purchase_timestamp       99441 non-null  object  
 8   order_approved_at              99281 non-null  object  
 9   order_delivered_carrier_date   97658 non-null  object  
 10  order_delivered_customer_date  96476 non